## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table = 'RPC_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table,
                'filter' : "AND item_sub_department_name = '400 RODENT PEST CONTROL' "})

snowflake_tools - 2025-06-25 19:57:40.931631+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-06-25 19:57:46.480715+10:00 - Saved token with '2025-06-25 23:57:46+10:00' expiry...


## Demographic Segments (AU)

In [6]:
RPC_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [18]:
# RPC_ds.head(10)

In [20]:
total_summary = RPC_ds.groupby('DEMOGRAPHIC_SEGMENT').agg(TOTAL_SALES_ALL_CLASS=('SEGMENT_SALES', 'first')).reset_index()
total_summary

,DEMOGRAPHIC_SEGMENT,TOTAL_SALES_ALL_CLASS
0,Homeowner older kids,482871.29
1,Homeowner younger kids,884279.52
2,Older homeowner,3842599.08
3,Renter no kids,302314.85
4,Renter with kids,295709.26
5,Unclassifiable,64457.95
6,Younger homeowner,711425.68


In [17]:
segment_index_summary = RPC_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SALES_INDEX', 'SEGMENT_SALES']]
segment_index_summary

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SALES_INDEX,SEGMENT_SALES
0,500 RODENT POISON,Homeowner older kids,0.935919,482871.29
1,500 RODENT POISON,Homeowner younger kids,0.843878,884279.52
2,500 RODENT POISON,Older homeowner,1.146328,3842599.08
3,500 RODENT POISON,Renter no kids,0.720799,302314.85
4,500 RODENT POISON,Renter with kids,0.708170,295709.26
5,500 RODENT POISON,Unclassifiable,1.129961,64457.95
6,500 RODENT POISON,Younger homeowner,0.919479,711425.68
7,500 RODENT REPELLENT,Homeowner older kids,0.958675,61015.46
8,500 RODENT REPELLENT,Homeowner younger kids,0.963838,124591.69
9,500 RODENT REPELLENT,Older homeowner,0.936824,387390.56


## Commercial Industry Segment (NZ)

In [10]:
# Get data
RPC_is = con.read_sql_file_into_pandas(
    file="sql/5-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': 10, 
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [11]:
total_summary = RPC_is.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('TOTAL_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('TOTAL_QUANTITY', 'first')
).reset_index()

In [12]:
segment_index_summary = RPC_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SALES_INDEX', 'QUANTITY_INDEX', 'CUSTOMER_INDEX']]
segment_index_summary

## Drop item range table

In [14]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table}
    )

# Brand view

In [15]:
overall_brand = con.read_sql_file_into_pandas(
    file="sql/Overall_by_brand.sql",
    formatting={
        "start_date":                 start_date,
        "end_date":                   end_date,
        "sub_dept":                   "'400 RODENT PEST CONTROL'"        
    })

In [16]:
overall_brand

,CUSTOMER_TYPE_CODE,BRAND_NAME,SALES,TOTAL_TRX,TOTAL_UNITS,NUM_ITEMS_PURCHASED,OVERALL_SALES,OVERALL_TRX
0,Consumer,BIG CHEESE,978429.28,40915,51017.0,26,1250863.29,50286
1,Consumer,KIWICARE,695580.01,31730,37769.0,12,940093.19,39491
2,Consumer,CLIX,0.22,1,1.0,1,0.22,1
3,Consumer,TRAPPED!,295159.09,24038,42779.0,9,373345.33,28896
4,Commercial,KIWICARE,244513.18,7761,9740.0,12,940093.19,39491
5,Commercial,NOOSKI,5341.51,337,457.0,3,26334.23,1752
6,Consumer,NO,6960.74,365,367.0,1,8275.91,437
7,Consumer,NOOSKI,20992.72,1415,1747.0,3,26334.23,1752
8,Commercial,GOTCHA,11099.77,1278,2066.0,3,63059.13,7913
9,Commercial,BIG CHEESE,272434.01,9371,12437.0,26,1250863.29,50286
